In [43]:
# requirements 
%pip install pandas --quiet 
%pip install flask requests --quiet
%pip install ollama --quiet

print("Requirements installed successfully!")


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Requirements installed successfully!


In [44]:
# imports
print("Starting imports...")
import pandas as pd
from flask import Flask, request, jsonify
import ollama
import json
import requests

print("Successfully imported!")

Starting imports...
Successfully imported!


In [45]:
# Load the dataset into a df
carsDF = pd.read_csv('vehicles.csv')

In [46]:
# view dataset 
carsDF.head()

,Type,Stock,VIN,Year,Make,Model,Body,ModelNumber,Doors,ExteriorColor,...,CityMPG,HighwayMPG,EPAClassification,Wheelbase_Code,Internet_Price,MarketClass,PassengerCapacity,ExtColorHexCode,IntColorHexCode,EngineDisplacementCubicInches
0,Used,T237890,A9FGFF8ST43CEFJ3Y,2019,Mazda,CX-9,Sport Utility,MACX-VF899A,4,Snowflake White Pearl Mica,...,20.0,26.0,Small SUV 4WD,115.3,0,4WD Sport Utility Vehicles,7,E7E8E8,NaN,152.0
1,Used,C120321,3RCKVP1GRJK1F6A8A,2024,Maserati,Levante,Sport Utility,MALE-SN743K,4,Bianco,...,16.0,22.0,Standard SUV 4WD,118.3,0,4WD Sport Utility Vehicles,5,NaN,NaN,182.0
2,Used,K666069,1W2XUCYU7BR72MDJR,2019,Volkswagen,Tiguan,Sport Utility,VOTI-UG651L,4,Platinum Gray Metallic,...,22.0,29.0,Small SUV 2WD,109.8,0,2WD Sport Utility Vehicles,7,NaN,NaN,121.0
3,Used,U309999,LDTMAMFH0Y2B3YXM1,2022,INFINITI,Q50,4dr Car,INQ5-CH685U,4,Midnight Black,...,20.0,26.0,Midsize Cars,112.2,0,4-door Mid-Size Passenger Car,5,08090B,NaN,183.0
4,Used,K436038,M4A5XPBCYH2HH2NXT,2019,Hyundai,Elantra,4dr Car,HYEL-AS987K,4,Symphony Silver,...,28.0,37.0,Midsize Cars,106.3,0,4-door Mid-Size Passenger Car,5,ADADB0,NaN,122.0


Data Cleaning and Preprocessing 

In [47]:
# Normalize text values to title case for consistency
carsDF['Make'] = carsDF['Make'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase
carsDF['Model'] = carsDF['Model'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase


In [48]:
# Verify normalization
print(carsDF[['Make', 'Model']])

            Make       Model
0          MAZDA        CX-9
1       MASERATI     LEVANTE
2     VOLKSWAGEN      TIGUAN
3       INFINITI         Q50
4        HYUNDAI     ELANTRA
...          ...         ...
2225      TOYOTA     COROLLA
2226       DODGE     DURANGO
2227  ALFA ROMEO      GIULIA
2228      NISSAN      SENTRA
2229      TOYOTA  TACOMA 2WD

[2230 rows x 2 columns]


In [49]:
# Data Type Correction
# Convert numerical columns
carsDF['Year'] = carsDF['Year'].astype(int)
carsDF['SellingPrice'] = carsDF['SellingPrice'].astype(float)
carsDF['Doors'] = carsDF['Doors'].astype(int)


In [50]:
carsDF['PriceRange'] = pd.cut(carsDF['SellingPrice'], 
                              bins=[0, 10000, 20000, 30000, 50000, 100000],
                              labels=['Under 10K', '10K-20K', '20K-30K', '30K-50K', 'Above 50K'])


In [54]:
def extract_preferences(message):
    """
    Function to extract user preferences from a message.
    Arguments:
    - message: User's message as a string.

    Returns:
    - preferences: A dictionary with extracted preferences.
    """
    preferences = {}

    if "new" in message.lower() or "used" in message.lower():
        preferences['car_type'] = 'New' if 'new' in message.lower() else 'Used'
    
    if 'year' in message.lower():
        year = [int(word) for word in message.split() if word.isdigit() and len(word) == 4]
        if year:
            preferences['year'] = year[0]

    if any(make in message.lower() for make in carsDF['Make'].str.lower().unique()):
        preferences['make'] = next(make.title() for make in carsDF['Make'].str.lower().unique() if make in message.lower())

    # Add more logic to extract model, price, etc.

    return preferences


In [55]:
@app.route('/follow_up', methods=['POST'])
def follow_up():
    conversation_id = request.json.get('conversation_id')

    response = ollama.chat(
        model="your-llama-model",
        message="Can you provide more details like your budget or preferred car features?",
        conversation_id=conversation_id
    )

    return jsonify({"reply": response['content'], "conversation_id": response['conversation_id']})


In [52]:
def search_cars(carsDF, car_type=None, year=None, make=None, model=None, model_number=None, stock=None, vin=None, certified=None):
    """
    Function to filter the cars DataFrame based on user-provided criteria.
    Arguments:
    - df: DataFrame containing car details.
    - car_type: 'New' or 'Used'
    - year: Year of manufacture (e.g., 2020)
    - make: Make of the car (e.g., 'Mazda')
    - model: Model of the car (e.g., 'CX-9')
    - model_number: Internal model code
    - stock: Dealer stock number
    - vin: Vehicle Identification Number
    - certified: Pre-owned certification status (True/False)
    
    Returns:
    - Filtered DataFrame based on provided criteria.
    """
    filtered_df = carsDF.copy()

    if car_type:
        filtered_df = filtered_df[filtered_df['Type'].str.upper() == car_type.upper()]
    if year:
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if make:
        filtered_df = filtered_df[filtered_df['Make'].str.upper() == make.upper()]
    if model:
        filtered_df = filtered_df[filtered_df['Model'].str.upper() == model.upper()]
    if model_number:
        filtered_df = filtered_df[filtered_df['ModelNumber'].str.upper() == model_number.upper()]
    if stock:
        filtered_df = filtered_df[filtered_df['Stock'].str.upper() == stock.upper()]
    if vin:
        filtered_df = filtered_df[filtered_df['VIN'].str.upper() == vin.upper()]
    if certified is not None:  # Certified can be True or False
        filtered_df = filtered_df[filtered_df['Certified'] == certified]

    return filtered_df



    Make Model  Year  Certified  SellingPrice
0  MAZDA  CX-9  2019      False       18498.0


In [56]:
def format_car_suggestions(cars):
    suggestions = ""
    for idx, car in cars.iterrows():
        suggestions += f"- {car['Year']} {car['Make']} {car['Model']}, priced at ${car['SellingPrice']:,.0f}. It has {car['Miles']} miles.\n"
    return f"Here are some options I found for you:\n{suggestions}Would you like to learn more about any of these cars?"


In [57]:
def get_car_details(car_id):
    car = carsDF[carsDF['VIN'] == car_id].iloc[0]
    return (f"This is a {car['Year']} {car['Make']} {car['Model']} with {car['Miles']} miles, "
            f"available for ${car['SellingPrice']:,.0f}. It comes with {car['Int_Upholstery']} upholstery "
            f"and has a {car['Fuel_Type']} engine.")


In [58]:
def compare_cars(vin1, vin2):
    car1 = carsDF[carsDF['VIN'] == vin1].iloc[0]
    car2 = carsDF[carsDF['VIN'] == vin2].iloc[0]
    
    comparison = (f"Comparing {car1['Make']} {car1['Model']} vs {car2['Make']} {car2['Model']}:\n"
                  f"- {car1['Make']} {car1['Model']} is priced at ${car1['SellingPrice']:,.0f} with {car1['Miles']} miles.\n"
                  f"- {car2['Make']} {car2['Model']} is priced at ${car2['SellingPrice']:,.0f} with {car2['Miles']} miles.\n"
                  f"{'The first car is more affordable.' if car1['SellingPrice'] < car2['SellingPrice'] else 'The second car is more affordable.'}")
    
    return comparison


In [60]:
def chat_with_llama2(user_message, conversation_id):
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama2",
        "message": user_message,
        "conversation_id": conversation_id
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        return response_data.get('content', "No response available")
    else:
        return f"Error: {response.status_code} - {response.text}"

In [59]:
# Flask App Definition
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    conversation_id = request.json.get('conversation_id')

    # Get response from Ollama model
    response_content = chat_with_llama2(user_message, conversation_id)

    # Use LLM to extract user preferences
    user_preferences = extract_preferences(response_content)
    filtered_results = search_cars(carsDF, **user_preferences)

    if filtered_results.empty:
        follow_up_message = "I couldn't find any cars matching your preferences. Would you like to modify your criteria?"
    else:
        top_cars = filtered_results.head(5)  # Get top 5 matches
        follow_up_message = format_car_suggestions(top_cars)

    # Respond with Ollama's response and filtered car suggestions
    return jsonify({"reply": follow_up_message, "conversation_id": conversation_id})

# Run the server
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
